In [1]:
import sys
import os

import torch
import pandas as pd
pd.set_option("display.max_colwidth", 400)
import tqdm


from pytorch_pretrained_bert import modeling, tokenization
from language_modeling.runners import (
    tokenize_example, InputExample,
    convert_example_to_features, features_to_data,
)
from glue.wnli_utils import masking_predictor

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
device = torch.device("cpu")
bert_model_name = "bert-base-uncased"

model = modeling.BertForPreTraining.from_pretrained(bert_model_name)
model.to(device);
model.eval();

tokenizer = tokenization.BertTokenizer.from_pretrained(bert_model_name, do_lower_case=True)

In [3]:
FOLDER = "../data"
WNLI_TRAIN_PATH = os.path.join(FOLDER, "train.tsv")
WNLI_DEV_PATH = os.path.join(FOLDER, "dev.tsv")
WNLI_TEST_PATH = os.path.join(FOLDER, "test.tsv")

train_df = pd.read_csv(WNLI_TRAIN_PATH, sep="\t")
val_df = pd.read_csv(WNLI_DEV_PATH, sep="\t")
test_df = pd.read_csv(WNLI_TEST_PATH, sep="\t")

In [9]:
STOP_AT = 10000

train_pred_arr, train_all_alt_ls, pos_a, pos_b, tok_a, tok_b = masking_predictor(train_df.iloc[:STOP_AT], tokenizer, 
                                                                                 model, device)

# val_pred_arr, val_all_alt_ls, *_ = masking_predictor(val_df, tokenizer, model)

print("Train acc: ", (train_pred_arr==train_df["label"][:STOP_AT]).mean())
# print("Val acc: ", (val_pred_arr==val_df["label"]).mean())

print("mean train pred:", train_pred_arr.mean())
# print("mean val pred:", val_pred_arr.mean())


Train acc:  0.5606299212598426
mean train pred: 0.6062992125984252


In [ ]:
filling_predictor(None, tokenizer, model)

## Visualize errors

In [ ]:
for i, (pred, true, sent1, sent2, faked) in enumerate(zip(train_pred_arr, train_df["label"], 
                                                   train_df['sentence1'], train_df['sentence2'], train_all_alt_ls)):
    if not (true == pred):
        print(sent1, sent2, faked[-1], bool(true), pred)
        print()
    if i > 80:    
        break